In [4]:
import functools
import itertools
import operator
from collections import defaultdict

import pandas as pd
import papermill as pm
import scrapbook as sb
from tqdm.auto import tqdm
from pathlib import Path

In [5]:
gpts = (256, 512, 1024, 2048)

In [6]:
name = "benchmark_name" # Please rename this

In [7]:
reports_path = Path(f"{name}-reports")
reports_path.mkdir(parents=True, exist_ok=True)

template_path = "prism_distributed_template.ipynb"

In [8]:
parameters = (gpts,)
total = functools.reduce(operator.mul, tuple(len(p) for p in parameters))

for (n,) in tqdm(itertools.product(*parameters), total=total):
    pm.execute_notebook(
        template_path,
        reports_path / f"prism_distributed-gpts={n}.ipynb",
        parameters=dict(gpts=n),
    )

  0%|          | 0/2 [00:00<?, ?it/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

Executing:   0%|          | 0/16 [00:00<?, ?cell/s]

In [15]:
notebooks = sb.read_notebooks(f"{name}-reports").notebooks

notebooks

index = []
data = defaultdict(list)
for nb in notebooks:
    index.append((nb.parameters["gpts"],))
    data["time_per_phonon"].append(nb.scraps.data_dict["time_per_phonon"])
    data["peak_memory"].append(nb.scraps.data_dict["peak_memory"])

index = pd.MultiIndex.from_tuples(index, names=["gpts"])
df = pd.DataFrame(data, index=index)
df = df.sort_index()

df

,time_per_phonon,peak_memory
gpts,,
256,0.839942,3.232156
512,5.016680,6.052173
